<a href="https://colab.research.google.com/github/gangoliya/gangoliya/blob/main/LinkedIn_problem_day54_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\gango\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Test').getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define the schema for the Weather DataFrame
weather_schema = StructType([
StructField("country_id", IntegerType(), True),
StructField("weather_state", IntegerType(),
True),
StructField("day", StringType(), True)
])
# Weather data
weather_data = [
(2, 15, "2019-11-01"),
(2, 12, "2019-10-28"),
(2, 12, "2019-10-27"),
(3, -2, "2019-11-10"),
(3, 0, "2019-11-11"),
(3, 3, "2019-11-12"),
(5, 16, "2019-11-07"),
(5, 18, "2019-11-09"),
(5, 21, "2019-11-23"),
(7, 25, "2019-11-28"),
(7, 22, "2019-12-01"),
(7, 20, "2019-12-02"),
(8, 25, "2019-11-05"),
(8, 27, "2019-11-15"),
(8, 31, "2019-11-25"),
(9, 7, "2019-10-23"),
(9, 3, "2019-12-23")
]

In [5]:
# Define the schema for the Countries DataFrame
countries_schema = StructType([
StructField("country_id", IntegerType(), True),
StructField("country_name", StringType(), True)
])
countries_data = [
(2, "USA"),
(3, "Australia"),
(7, "Peru"),
(5, "China"),
(8, "Morocco"),
(9, "Spain")
]

In [6]:
weather_df = spark.createDataFrame(weather_data, weather_schema)
weather_df.show()

Py4JJavaError: An error occurred while calling o38.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (Gangoliya executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.colle

In [ ]:
countries_df = spark.createDataFrame(countries_data, countries_schema)
countries_df.show()

+----------+------------+
|country_id|country_name|
+----------+------------+
|         2|         USA|
|         3|   Australia|
|         7|        Peru|
|         5|       China|
|         8|     Morocco|
|         9|       Spain|
+----------+------------+



In [ ]:
df_combined = countries_df.join(weather_df, on="country_id")


df_filtered = df_combined.filter((df_combined.day >= "2019-11-01")&(df_combined.day <= "2019-11-30"))

In [ ]:
from pyspark.sql.functions import avg
df_groupped = df_filtered.groupBy("country_name").agg(avg("weather_state").alias("average"))

In [ ]:
from pyspark.sql.functions import when

df_final = df_groupped.withColumn("weather_type",when(df_groupped.average <= 15,"COLD")\
                                                  .when(df_groupped.average >= 25,"HOT")\
                                                  .otherwise("WARM")
                       ).select("country_name","weather_type").show()

+------------+------------+
|country_name|weather_type|
+------------+------------+
|        Peru|         HOT|
|       China|        WARM|
|     Morocco|         HOT|
|         USA|        COLD|
|   Australia|        COLD|
+------------+------------+

